Google Colab에서 Google Drive를 마운트해서 드라이브 내 파일들에 접근 허용
`with open('경로', 'r') as _ : ` 등으로 파일 읽기 가능

In [10]:
# 계산, 통계에 자주 사용되는 Numpy 라이브러리
import numpy as np

In [11]:
# Python 정규표현식 모듈
import re

## 1. 입력데이터 전처리 작업
**1.한글, 숫자, 영어를 제외한 글자는 모두 '_'로 치환**

[^...] : 부정 문자 클래스, 괄호 안의 문자들이 아닌 것들을 매칭

0-9 : 아라비아 숫자 (0~9)

a-zA-Z : 영어 알파벳 (소문자 + 대문자)

\u3131-\u3163 : 한글 자모 범위 (ㄱ ~ ㅣ)

\uac00-\ud7a3 : 완성형 한글 (가 ~ 힣)

**2. 언더바가 하나 이상일 경우 언더바 하나로 치환**

In [12]:
def preprocess_text(text):

    """
    input_text = '아 이 영화 스토리랑 연기랑;; 개ㅐㅐ노잼^^!!!!! 인데 왜 9점임???'
    output_text = '$아_이_영화_스토리랑_연기랑_개ㅐㅐ노잼_인데_왜_9점임_$'
    """

    non_alpha_numeric_hangul = re.compile('[^0-9a-zA-Z\u3131-\u3163\uac00-\ud7a3]')
    sentense_separator = '$'

    # text의 대상 문자열을 _ 로 변경
    t = non_alpha_numeric_hangul.sub('_', text)

    # _가 2개 이상일 경우, 하나로 치환
    text = re.sub(r'_+', '_', t)
    return f'{sentense_separator}{text}{sentense_separator}'

# 2. 입력 파일 형식에 맞춰 data와 target 값 읽어오기
---

**id	document	label**<br>
6270596	굳 ㅋ	1 <br>
9274899	GDNTOPCLASSINTHECLUB	0 <br>
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0<br>
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0

In [13]:
def prepare_data_file(FILE_PATH):

    # 경로로 파일 열기
    with open(FILE_PATH) as f:
        lines = f.read().split('\n') # 한 줄씩 읽어오기

    data, target = [], []
    for l in lines[1:]: # 첫째 줄은 컬럼명이라 읽어오지 않음
        try:
            _, text, label = l.strip().split('\t') # 탭으로 구분
        except ValueError:
            pass
        text = text.strip() # text 내부의 공백 제거
        if text == '' : continue # 공백 데이터는 학습 데이터로 사용하지 않음
        data.append(preprocess_text(text))
        target.append(int(label))

    return data, target

# 3. Bigram feature를 추출하여 feature_dict으로 만들어서 리턴
MAX_FEATURES : 사용할 feature 개수

_feature_dict는 ML 벡터화에 쓰이는 biagram -> 숫자 맵핑테이블_

In [14]:
def extract_features(data, MAX_FEATURES):

    FEATURES = dict()

    """
    bigram을 생성하고 등장 빈도수 체크
    """

    for line in data:
        uni_list = list(line)
        bi_list = [''.join(uni_list[z:z+2]) for z in range(0, len(uni_list)-1)]

        for bigram in bi_list:

            if bigram in FEATURES: # 빈도수 누적
                FEATURES[bigram] += 1
            else:
                FEATURES[bigram] = 1

    features_list = [(b, c) for (b, c) in FEATURES.items()]
    features_list.sort(reverse=True, key=lambda z:z[1]) # count 순으로 내림차순 정렬

    features_dict = dict()
    for (idx, (b, c)) in enumerate(features_list[:MAX_FEATURES]):
        features_dict[b] = idx

    return features_dict


# 4. 입력 문장을 고정된 크기의 Feature  Vector로 변환하여 ML 학습에 사용할 입력 벡터 생성
Bigram 기반의 이진 벡터

In [15]:
def make_feature_vector(feature_set, data, target):

    # 기본 벡터 템플릿 생성 [0,0,0,0, ..., 0]
    fv_base = [0 for _ in range(0, len(feature_set))]
    feature_list = []

    for (x, label) in zip(data, target):
        uni_list = list(x)
        fv = fv_base[:]
        bi_list = [''.join(uni_list[z:z+2]) for z in range(0, len(uni_list)-1)]

        # one-hot 방식
        for bigram in bi_list:
            if bigram in feature_set:
                fv[feature_set[bigram]] = 1
        feature_list.append(fv + [label])
    feature_list = np.array(feature_list)

    # ML시 train/test bias를 방지하기 위해 셔플
    np.random.shuffle(feature_list)

    return feature_list[:, :-1], feature_list[:, -1]

## 학습데이터, 평가데이터 읽어오기

In [16]:
TRAIN_FILE = 'ratings_train.txt'
TEST_FILE  = 'ratings_test.txt'

print('prepare_data_file START...')
train_data, train_target = prepare_data_file(TRAIN_FILE)
test_data, test_target = prepare_data_file(TEST_FILE)
print('prepare_data_file END...')

prepare_data_file START...
prepare_data_file END...


## 학습데이터로부터 MAX_FEATURES개의 Bigram Feature 추출

In [17]:
print('extract_features START...')
MAX_FEATURES = 1000
feature_set = extract_features(train_data, MAX_FEATURES)
with open('features.out', 'w', encoding='utf-8') as fo:
    fo.write('\n'.join([x+str(idx) for x, idx in feature_set.items()]))

print('extract_features END...')


extract_features START...
extract_features END...


## 입력 파일을 고정된 크기의 feature vector로 변환

In [18]:
print('make_feature_vector START...')
x_train, y_train = make_feature_vector(feature_set, train_data, train_target)
x_test, y_test = make_feature_vector(feature_set, test_data, test_target)
print('make_feature_vector END...')

make_feature_vector START...
make_feature_vector END...


## ML 모델 생성 및 학습

In [21]:
from sklearn.naive_bayes import MultinomialNB

print('train START...')
model = MultinomialNB()
model.fit(x_train, y_train)
print('train END...')

train START...
train END...


## 정확도 평가

In [22]:
print('eval START...')
print(f'TRAIN ACCURACY : {model.score(x_train, y_train):3f}')
print(f'TEST SCORE : {model.score(x_test, y_test):3f}')
print('evel END...')

eval START...
TRAIN ACCURACY : 0.780614
TEST SCORE : 0.777651
evel END...


## 실행

In [23]:
# 예측 실패
text = '어쩌다 이 영화를 보게 되었는지.. 함꼐 본 사람은 좋았지만 영화는 그를 잊을만큼 정말 별로였어요'
text = preprocess_text(text)
x_test, _ = make_feature_vector(feature_set, [text], [None])
result = model.predict(x_test)
print(text, '==>', ['Negative','Positive'][result[0]])

$어쩌다_이_영화를_보게_되었는지_함꼐_본_사람은_좋았지만_영화는_그를_잊을만큼_정말_별로였어요$ ==> Positive


In [24]:
# 예측 성공
text = '연기도 재밌고 존잼!'
text = preprocess_text(text)
x_test, _ = make_feature_vector(feature_set, [text], [None])
result = model.predict(x_test)
print(text, '==>', ['Negative','Positive'][result[0]])

$연기도_재밌고_존잼_$ ==> Positive


In [25]:
# 예측 성공
text = 'ㅈㄴ별로임 진짜... 하.... 돈 아까워'
text = preprocess_text(text)
x_test, _ = make_feature_vector(feature_set, [text], [None])
result = model.predict(x_test)
print(text, '==>', ['Negative','Positive'][result[0]])

$ㅈㄴ별로임_진짜_하_돈_아까워$ ==> Negative
